In [3]:
#instalação das bibliotecas necessárias
#pip install selenium
#pip install pandas
#pip install matplotlib


  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
#importação das bibliotecas
from selenium.webdriver import Chrome
import time as t
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#importa o chromedriver
#acessa o site que vai fornecer a unidade federativa e a população

In [ ]:
navegador = Chrome("chromedriver")
navegador.get("http://www.bibliotecavirtual.sp.gov.br/temas/sao-paulo/sao-paulo-populacao-do-estado.php")
navegador.maximize_window()
t.sleep(3)

In [ ]:
#encontra os elementos para encontrar a tabela com as informações

In [ ]:

linhas = navegador.find_elements_by_xpath("/html/body/div[3]/div/div[1]/article/div/div")

In [ ]:
#percorre as linhas encontradas em busca das tags "td"
#adiciona as informações "unidade federativa" e "população" em uma lista

In [ ]:
lista = []
for linha in linhas:
    tds = linha.find_elements_by_tag_name('td')
    for i, t in enumerate(tds, 1):
        lista.append(t.text)

In [ ]:
#cria um dicionário com a sigla e o nome das unidades federativa, para facilitar a inclusão dos dados

In [3]:
ufs = {"AC": "Acre", "AL": "Alagoas", "AM": "Amazonas", "AP": "Amapá", "BA": "Bahia", "CE": "Ceará",
       "DF": "Distrito Federal", "ES": "Espirito Santo", "GO": "Goiás", "MA": "Maranhão", "MG": "Minas Gerais",
       "MS": "Mato Grosso do Sul", "MT": "Mato Grosso", "PA": "Pará", "PB": "Paraiba", "PE": "Pernambuco",
       "PI": "Piauí", "PR": "Paraná", "RJ": "Rio de Janeiro", "RN": "Rio Grande do Norte", "RO": "Rondônia",
       "RR": "Roraima", "RS": "Rio Grande do Sul", "SC": "Santa Catarina", "SE": "Sergipe", "SP": "São Paulo",
       "TO": "Tocantins"}

In [ ]:
# 1 - percorre todas as unidades federativas
# 2 - para cada uma inclui sigla na url e faz a leitura do csv sem baixar o arquivo
# 3 - pega a quantidade de linhas do csv para a quantidade de vacinados
# 4 - percorre a lista de população e pega a quantidade da unidade atual
# 5 - calcula a porcentagem da população vacinada
# 6 - monta os dados da planilha a ser criada
# 7 - monta cabeçalho
# 8 - plota a planilha

In [ ]:
for uf in ufs:
    url = f'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-07-06/uf%3D{uf}/'\
          f'part-00000-2266a6be-a6af-4755-86bc-9c3b75d8f506.c000.csv'

    df = pd.read_csv(url, encoding='cp1252', skiprows=1, sep=';', low_memory=False)
    vacinados = len(df.index)
    populacao = ''

    for index, linha in enumerate(lista):
        if ufs[uf] in linha:
            populacao = lista[index+1]
            populacao = str(populacao).replace('.', '')
            break

    aderencia = (100 * vacinados) / int(populacao)
    fig, ax = plt.subplots(1, 1)

    data = [[ufs[uf], populacao, f'{vacinados:,.2f}', '{:.2f}'.format(aderencia)]]

    column_labels = ["Unidade Federativa", "População", "Vacinados", "Aderência (%)"]
    ax.axis('tight')
    ax.axis('off')
    ax.table(cellText=data, colLabels=column_labels, loc="center")

    plt.show()